In [1]:
import numpy as np
import pandas as pd

from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder, label_binarize
from sklearn.pipeline import Pipeline

2021-07-15 19:55:25.812127: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
abs_path = '/home/ssc/Entwicklung/projects/arguseyes/arguseyes/example_pipelines/datasets/sneakers/'

def decode_image(img_str):
    return np.array([int(val) for val in img_str.split(':')])

train_data = pd.read_csv(f'{abs_path}product_images.csv', converters={'image': decode_image })

In [3]:
product_categories = pd.read_csv(f'{abs_path}product_categories.csv')

In [4]:
with_categories = train_data.merge(product_categories, on='category_id')

categories_to_distinguish = ['Sneaker', 'Ankle boot']

images_of_interest = with_categories[with_categories['category_name'].isin(categories_to_distinguish)]

In [13]:
def normalise_image(images):
    return images / 255.0

def reshape_images(images):
    return np.concatenate(images['image'].values) \
             .reshape(images.shape[0], 28, 28, 1)

def create_cnn():
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu',
                                     input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

pipeline = Pipeline(steps=[
    ('normalisation', FunctionTransformer(normalise_image)),
    ('reshaping', FunctionTransformer(reshape_images)),
    ('model', KerasClassifier(create_cnn))
])

In [14]:
train, test = train_test_split(images_of_interest, test_size=0.2)

y_train = label_binarize(train['category_name'], classes=categories_to_distinguish)
y_test = label_binarize(test['category_name'], classes=categories_to_distinguish)

In [15]:
model = pipeline.fit(train[['image']], y_train)

30/30 [==============================] - 1s 23ms/step - loss: 0.3016 - accuracy: 0.8726


In [17]:
model.score(test[['image']], y_test)

8/8 [==============================] - 0s 6ms/step - loss: 0.2026 - accuracy: 0.9160


0.9159663915634155